In [1]:
import numpy as np
import random
import math

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
Lengt = 1
dx = 0.01
Time = 20
dt = 0.01

umax = 0.005

TN = (int)(Time/dt)
Nx = (int)(Lengt/dx)

z = np.zeros(shape=(TN, Nx))
"""
for i in range(Nx):
    if i < int(Nx / 2):
        z[0][i] = i * dx * umax / (Lengt/2)
    elif i >= int(Nx / 2):
        z[0][i] = (Nx - i) * dx * umax / (Lengt/2)
"""
"""
for i in range(Nx):
  if i > int(Nx / 4) and i < int(Nx / 2):
    z[0][i] = (i - int(Nx / 4)) * dx * umax / (Lengt/2)
  elif i >= int(Nx / 2) and i < int(Nx / 4 * 3):
    z[0][i] = (Nx - i - int(Nx / 4)) * dx * umax / (Lengt/2)
"""
"""
for i in range(Nx):
  if i > int(Nx / 4) - int(Nx/5) and i < int(Nx / 2) - int(Nx/5):
    z[0][i] = (i - int(Nx / 4) + int(Nx/5)) * dx * umax / (Lengt/2)
  elif i >= int(Nx / 2) - int(Nx/5) and i < int(Nx / 4 * 3) - int(Nx/5):
    z[0][i] = (Nx - i - int(Nx / 4) - int(Nx/5)) * dx * umax / (Lengt/2)
"""

for i in range(Nx):
  if i >= int(Nx / 10 * 2) and i < int(Nx / 10 * 3):
    z[0][i] = umax / 5
  elif i >= int(Nx / 10 * 3) and i < int(Nx / 10 * 4):
    z[0][i] = umax / 5 * 2
  elif i >= int(Nx / 10 * 4) and i < int(Nx / 10 * 5):
    z[0][i] = umax / 5
  else:
    z[0][i] = 0


In [ ]:
x = np.linspace(0, Lengt, Nx)

plt.figure(figsize=(15, 6)) 
plt.plot(x, z[0])

## Изначальная формула
![title](Formula.png)
## Формула для нахождения U(i+1,j)
![title](Formula2.png)

In [ ]:
O = 2

a = dt / dx / O

print(a)
a = a**2

for t in range(1, TN):
    for i in range(1, Nx-1):
        if t > 1:
            z[t][i] = a * (z[t - 1][i + 1] - 2*z[t - 1][i] + z[t - 1][i - 1]) + 2*z[t-1][i] - z[t - 2][i]
        else:
            z[t][i] = z[0][i]


In [ ]:
fig = go.Figure(data=[go.Surface(z=z, x=x, y=np.linspace(0, Time, TN), colorscale = 'Rainbow')])

fig.update_layout(title='Struna',
                  autosize=False,
                  width=1000, height=1000)

fig.update_layout(scene = dict(zaxis = dict(range=[-0.02,0.02],),))


fig.update_layout(width=800, height=800)
fig.show()

#Результат
![](Plot1.png)

In [ ]:
frames = []
for i in range(TN):
  frames.append(go.Frame(data=[go.Scatter(x=x, y=z[i])]))

In [ ]:
fig = go.Figure(data=[go.Scatter(x=np.linspace(0, Lengt, (int)(Lengt/dx)), y=z[0])],
                layout=go.Layout(
                    xaxis=dict(range=[0, Lengt], autorange=False),
                    yaxis=dict(range=[-0.01, 0.013], autorange=False),
                    title="Start Title",
                    updatemenus=[dict(
                        type="buttons",
                        buttons=[dict(label="Play",
                                      method="animate",
                                      args=[None, {"frame": {"duration": 10, "redraw": False},
                                                   "fromcurrent": True, "transition": {"duration": 0,
                                                                                       "easing": "quadratic-in-out"}}]),
                                 dict(label="Pause",
                                      method="animate",
                                      args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                     "mode": "immediate",
                                                     "transition": {"duration": 0}}]

                                      )])]
                ),
                frames=frames
                )

fig.update_layout(title='Struna', autosize=False,
                  width=1000, height=500)

fig.show()

## Тут будет выведена анимация
## Ниже сохранение этой анимации в график

In [ ]:
import gif

@gif.frame
def plot(i):
    f = go.Figure(data=[go.Scatter(x=x, y=z[i])],
                  layout=go.Layout(
                      xaxis=dict(range=[0, Lengt], autorange=False),
                      yaxis=dict(range=[-0.01, 0.01], autorange=False)))
    return f

# Construct list of frames
frames = []
for i in range(TN):
    frame = plot(i)
    frames.append(frame)

gif.save(frames, 'Gif1.gif', duration=10)

## Процесс колебания
![](ezgif.com-gif-maker.gif)

In [ ]:
sound = np.zeros(TN)
for i in range(1, TN):
    for j in range(Nx):
        sound[i] += ((z[i][j] - z[i - 1][j]) / dt)**2
        
plt.figure(figsize=(15, 10)) 
plt.plot(np.linspace(0, Time, TN), sound)

In [ ]:
from scipy.fft import fft, fftfreq
F_result = fft(sound)
F_result

xf = fftfreq(TN, dt)[:TN//2]
plt.figure(figsize=(15, 10)) 
plt.xlim(0,35)
plt.ylim(0,0.0003)
plt.plot(xf, 2.0/TN * np.abs(F_result[0:TN//2]))
plt.grid()
plt.show()

In [ ]:
frames = []
for i in range(TN):
  frames.append(go.Frame(data=[go.Scatter(x=x, y=z[i],
    marker = dict(size = sound[i]))]))